# Application of Bootstrap samples in Random Forest

In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm_notebook

 <li> Load the boston house dataset </li>

In [2]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable
y=y.reshape(-1,1)

In [3]:
x.shape

(506, 13)

### Task: 1
<font color='red'><b>Step 1 Creating samples: </b></font> Randomly create 30 samples from the whole boston data points.
<ol>
<li>Creating each sample: Consider any random 303(60% of 506) data points from whole data set and then replicate any 203 points from the sampled points</li>
<li>Ex: For better understanding of this procedure lets check this examples, assume we have 10 data points [1,2,3,4,5,6,7,8,9,10], first we take 6 data points randomly consider we have selected [4, 5, 7, 8, 9, 3] now we will replciate 4 points from [4, 5, 7, 8, 9, 3], consder they are [5, 8, 3,7] so our final sample will be [4, 5, 7, 8, 9, 3, 5, 8, 3,7]</li>
<li> we create 30 samples like this </li>
<li> Note that as a part of the Bagging when you are taking the random samples make sure each of the sample will have                different set of columns</li>
<li> Ex: assume we have 10 columns for the first sample we will select [3, 4, 5, 9, 1, 2] and for the second sample [7, 9, 1, 4, 5, 6, 2] and so on...</li>
<li> Make sure each sample will have atleast 3 feautres/columns/attributes</li>
</ol>

<font color='red'><b>Step 2 Building High Variance Models on each of the sample and finding train MSE value:</b></font> Build a DecisionTreeRegressor on each of the sample.
<ol><li>Build a regression trees on each of 30 samples.</li>
<li>computed the predicted values of each data point(506 data points) in your corpus.</li>
<li> predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{30}\sum_{k=1}^{30}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.</li>
<li>Now calculate the $MSE =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.</li>
</ol>

<font color='red'><b>Step 3 Calculating the OOB score :</b></font>
<ol>
<li>Computed the predicted values of each data point(506 data points) in your corpus.</li>
<li>Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{k}\sum_{\text{k= model which was buit on samples not included } x^{i}}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.</li>
<li>Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.</li>
</ol>

### Task: 2
<pre>
<font color='red'><b>Computing CI of OOB Score and Train MSE</b></font>
<ol>
<li> Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score </li>
<li> After this we will have 35 Train MSE values and 35 OOB scores </li>
<li> using these 35 values (assume like a sample) find the confidence intravels of MSE and OOB Score </li>
<li> you need to report CI of MSE and CI of OOB Score </li>
<li> Note: Refer the Central_Limit_theorem.ipynb to check how to find the confidence intravel</li>
</ol>
</pre>
### Task: 3
<pre>
<font color='red'><b>Given a single query point predict the price of house.</b></font>

<li>Consider xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60] Predict the house price for this point as mentioned in the step 2 of Task 1. </li>
</pre>

In [4]:
#https://codereview.stackexchange.com/questions/193835/checking-a-one-dimensional-numpy-array-in-a-multidimensional-array-without-a-loo

def check(test,array):
    return any(np.array_equal(x, test) for x in array)

In [5]:
#https://stackoverflow.com/questions/14262654/numpy-get-random-set-of-rows-from-2d-array


#CREATE SAMPLES 30 TIMES WHERE EACH SAMPLE HAS 506 SAMPLES
def createsample(x,y):
    lx=[]
    ly=[]
    col=[]
    
    
    for i in (range (30)):
        #RANDOMLY SAMPLE 303 DATAPOINTS WITHOUT REPLCAEMENT FROM X AND Y AND STORE IT IN TEMP AND TEMP_Y
        rowind=np.random.choice(x.shape[0], 303,replace=False)
        colind=np.random.choice(x.shape[1],size=np.random.randint(low=3,high=13),replace=False)
        temp=x[:,colind][rowind,:]
        temp_y= y[rowind,:]

        #RADOMLY SAMPLE 203 DATAPOINTS WITH REPLCAMENT FROM TEMP AND TEMP_Y AND STORE IT IN TEMP1 AND TEMP1_Y
        rowind_replaced= np.random.choice(temp.shape[0], 203, replace=True)
        temp1=temp[rowind_replaced,:]
        temp1_y=temp_y[rowind_replaced,:]
        
        #CONCATENATE TEMP AND TEMP1 TO CREATE SAMPLE_X
        #CONCATENATE TEMP_Y AND TEMP1_Y TO CREATE SAMPLE_Y
        sample_x=np.vstack((temp,temp1))
        sample_y=np.vstack((temp_y,temp1_y))
        
        lx.append(sample_x)
        ly.append(sample_y)
        col.append(colind)
        
    return(lx,ly,col)

In [6]:
def average_y(lx,x,col):

    pred=[]
    pred1=[]
    
    #FOR EVERY ith DATAPOINT IN WHOLE DATASET
    for item in (x):
        s=0 #SUM FOR MSE FOR WHOLE DATA
        s1=0 #SUM FOR MSE FOR OOB POINTS
        cnt=0 #KEEPINNG COUNT THE NUMBER OF POINTS THAT IS NOT PRESENT IN THE SAMPLED DATA

        # FOR EVERY SAMPLED DATAPOINT
        for j in range (0,len(lx)):
            #FIT THE DATA
            clf=DecisionTreeRegressor()
            clf.fit(lx[j],ly[j])
            t=item[col[j]]
            
            #CALCULATE YPRED FO WHOLE DATA
            ypred=clf.predict(t.reshape(1,-1))
            s=s+ypred
            
            #IF ith point NOT IN THE SAMPLED DATA
            
            if not check(t,lx[j]):
                #CALCULATE YPRED FOR OOB
                ypred1=clf.predict(t.reshape(1,-1))
                #print(ypred)
                s1=s1+ypred1
                cnt+=1
                

        pred.append(s/30)#y_pred for whole data
        pred1.append(s1/cnt)

    
    pred=np.array(pred)
    pred1=np.array(pred1)
    return(pred,pred1)
        
        
        
    

In [7]:
lx,ly,col=createsample(x,y)#FUNCTION CREATES 30 SAMPLES EACH OF LENGTH 506 AND IS SAMPLED AS MENTIONED IN THE INSTRUCTIONS

pred,pred1=average_y(lx,x,col)#FUNCTION RETURNS THE AVEREAGE VALUE OF OF YPREDCITED FOR WHOLE DATASET AS WELL AS OOB PTS




oob=mean_squared_error(pred1,y)
mse=mean_squared_error(y,pred)

print("MSE for whole data",mse)
print("MSE for OOB pts",oob)


MSE for whole data 2.3207908794953225
MSE for OOB pts 13.577434028426964


### TASK 2

In [8]:
oob=[]
mse=[]
for i in tqdm_notebook(range(0,35)):
    
    lx,ly,col=createsample(x,y)
    pred,pred1=average_y(lx,x,col)
    

    
    
    ob=mean_squared_error(pred1,y)
    ms=mean_squared_error(y,pred)
    
    oob.append(ob)
    mse.append(ms)

In [9]:
mse=np.array(mse)
oob=np.array(oob)

In [10]:
sample_mean = mse.mean()
sample_std =  mse.std()
sample_size = len(mse)
left_limit  = np.round(sample_mean - 2*(sample_std/np.sqrt(sample_size)), 3)
right_limit = np.round(sample_mean + 2*(sample_std/np.sqrt(sample_size)), 3)

print("95 % CI for population mean of MSE for OOB pts is",left_limit,"-",right_limit)

95 % CI for population mean of MSE for OOB pts is 2.414 - 2.637


In [11]:
sample_mean = oob.mean()
sample_std =  oob.std()
sample_size = len(oob)
left_limit  = np.round(sample_mean - 2*(sample_std/np.sqrt(sample_size)), 3)
right_limit = np.round(sample_mean + 2*(sample_std/np.sqrt(sample_size)), 3)

print("95 % CI for population mean of oob for whole data is",left_limit,"-",right_limit)

95 % CI for population mean of oob for whole data is 14.028 - 14.711


## TASK 3

In [12]:
lx,ly,col=createsample(x,y)
xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60]
xq=np.array(xq)

In [13]:
s=0

for j in range (0,len(lx)):
    clf=DecisionTreeRegressor()
    clf.fit(lx[j],ly[j])
    t=xq[col[j]]

    ypred=clf.predict(t.reshape(1,-1))
    s=s+ypred
    
print("the predited house price for given query point is",s/30)

the predited house price for given query point is [19.62333333]
